In [1]:
## Standard Imports ##
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import numpy as np
import scipy as sp

import time as timeit
import datetime
import sys,os

## Particle Filter Imports ##
import particles
from particles import state_space_models as ssm
from particles import smc_samplers as ssp
from particles import distributions as dists
import warnings; warnings.simplefilter('ignore')  # hide warnings 

## Compiling the Particle Filter Example
Referencing the description to ```03-PF_Example```, the particle filter scheme used in this illustrative study developes a convergent posterior model for the majority of cases using the parameters ```init_Nx = 10``` and ```N = 50```. For those cases which are not convergent under those conditions, the parameters ```init_Nx = 15``` and ```N = 75``` were used. 

The purpose of this code is then to combine the outputs from these disparate starting conditions to develop a complete example case which includes results for all 50 prior distributions on the parameters. 

In [ ]:
### Lay Out The Files Holding the PF Results ###
pfOutFiles = np.array(['../04-Data/Bouc-Wen/outputPF_Nx10_Ntheta50.npz',
                       '../04-Data/Bouc-Wen/outputPF_Nx15_Ntheta75.npz'])
saveFile = '02 - Results/outputPF'

### Generate Storage Over Inferred States/Parameters ###
outData = np.load(pfOutFiles[0])
muHist = np.zeros(outData['muHist'].shape)
    # mean of the inferred parameters for each inference trial
    # over the observation period. This is what the EKF directly
    # outputs
stdHist = np.zeros(outData['stdHist'].shape)
    # standard deviation of the inferred parameters for each 
    # inference trial over the observation period. This is 
    # what the EKF directly outputs
meanHist = np.zeros(outData['meanHist'].shape)
    # mean of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modeHist = np.zeros(outData['modeHist'].shape)
    # mode of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modStates = np.zeros(outData['modStates'].shape)
    # Response history of the inferred system given the input
    # excitation. Essentially, we're remodeling the behavior of 
    # the system given our selections on point estimates of the 
    # parameters from the posterior. 
runTime = np.zeros(outData['runTime'].shape)
    # Computational time for each inference trial. 
Nx = np.zeros(outData['Nxs'].shape)
    # Tracks the increase in number of particles on the parameters
    # over the inference history. 

### Combine Results Files ###
for i in range(len(pfOutFiles)):
    outData = np.load(pfOutFiles[i])

    for j in range(outData['muHist'].shape[0]):
        ## If the trial failed to converge, do not include in compiled results
        if outData['Nxs'][j,-1] == 0:
            continue
        ## If the trial converged, include in compiled results
        else:
            print('Inserting Data from ' + pfOutFiles[i] + '\n\t@ Iteration %d:'%(j))

            muHist[j,:,:] = outData['muHist'][j,:,:]         # inference history of untransformed state/par means
            stdHist[j,:,:] = outData['stdHist'][j,:,:]       # inference history of state/par standard deviations
            meanHist[j,:,:] = outData['meanHist'][j,:,:]     # inference history of transformed state/par means
            modeHist[j,:,:] = outData['modeHist'][j,:,:]     # inference history of transformed state/par modes
            modStates[j,:,:] = outData['modStates'][j,:,:]   # states that have been remodeled based on the final modes of the parameters
            Nx[j,:] = outData['Nxs'][j,:]                  # Number of particle sample runs
            runTime[j] = outData['runTime'][j]           # Run Time

            print('\tComputation Time = %d minutes and %d seconds' %(np.floor(runTime[j]), 
                                                            np.floor((runTime[j]*60) - 60*np.floor(runTime[j]))) )
            print('\tMode of Final Parameter Distributions: \n\t\txi = %.4f,\n\t\twn = %.4f\n'
              %(modeHist[j,2,-1],modeHist[j,3,-1]))
            
### Save Results ###
np.savez(saveFile, muHist = muHist,stdHist=stdHist, meanHist=meanHist, modeHist=modeHist, 
         modStates = modStates, Nx = Nx, runTime = runTime)

## Load Experimental Data

In [27]:
### Generate Storage Over States ###
parPriors = np.loadtxt('Priors_lognormal.txt')
nInf = 8
nState = states.shape[0]
nPar = nInf - nState
samps = len(time)

muHist = np.zeros((parPriors.shape[0],nInf, samps))
stdHist = np.zeros((parPriors.shape[0],nInf, samps))
meanHist = np.zeros((parPriors.shape[0],nInf, samps))
modeHist = np.zeros((parPriors.shape[0],nInf, samps))
modStates = np.zeros((parPriors.shape[0],nInf, samps))
Nx = np.zeros((parPriors.shape[0],samps))
runTime = np.zeros(parPriors.shape[0])

for j in range(len(parPriors)):
    print('Inserting Data from Nx=10, N=50 @ Iteration %d:'%(j))
    try: 
        outFile = '05 - Results/PF Runs/outputPF_secondRun_Nx10_N50_' + str(j) + '.npz'
        outData = np.load(outFile)

        muHist[j,:,:] = outData['muHist']         # inference history of untransformed state/par means
        stdHist[j,:,:] = outData['stdHist']       # inference history of state/par standard deviations
        meanHist[j,:,:] = outData['meanHist']     # inference history of transformed state/par means
        modeHist[j,:,:] = outData['modeHist']     # inference history of transformed state/par modes
        modStates[j,:,:] = outData['modStates']   # states that have been remodeled based on the final modes of the parameters
        Nx[j,:] = outData['Nxs']                  # Number of particle sample runs
        runTime[j] = outData['runTime']           # Run Time

    except:
        print('\n\tERROR: Instability or Run Incomplete\n')
        ## Store Results ##
        for i in range(1,samps):
            muHist[j,:,i] = np.array([0,0,0, -0.1054, 2.3, 3.4, 1.9, 3.4])
            stdHist[j,:,i] = 0.01*np.ones(8)
            meanHist[j,:,i] = np.concatenate((muHist[j,:nState,i], 
                                    np.exp(muHist[j,nState:,i] + np.square(stdHist[j,nState:,i])/2)))
            modeHist[j,:,i] = np.concatenate((muHist[j,:nState,i], np.exp(muHist[j,nState:,i] 
                                    - np.square(stdHist[j,nState:,i]))))

        ### Rerun Model with Identified Parameters ###
        modStates[j,:,0] = np.concatenate((np.zeros((3,)), np.log(modeHist[j,3:,-1])))
        for i in range(1,samps):
            modStates[j,:,i] = fx(modStates[j,:,i-1], dt, exc=inpAcc[i-1]) 
            
    print('\tComputation Time = %d minutes and %d seconds' %(np.floor(runTime[j]), 
                                                        np.floor((runTime[j]*60) - 60*np.floor(runTime[j]))) )
    print('\tMode of Final Parameter Distributions: \n\t\txi = %.4f,\n\t\twn = %.4f,\n\t\tbeta = %.4f,\n\t\tn = %.4f,\n\t\tgamma = %.4f\n'
          %(modeHist[j,3,-1],modeHist[j,4,-1],modeHist[j,5,-1],modeHist[j,6,-1],modeHist[j,7,-1]))
            
        
for j in range(len(parPriors)):
    if Nx[j,0] == 0:
        print('Inserting Data from Nx=15, N=75 @ Iteration %d:'%(j))
        try: 
            outFile = '05 - Results/PF Runs/outputPF_Nx15_N75_' + str(j) + '.npz'
            outData = np.load(outFile)

            muHist[j,:,:] = outData['muHist']         # inference history of untransformed state/par means
            stdHist[j,:,:] = outData['stdHist']       # inference history of state/par standard deviations
            meanHist[j,:,:] = outData['meanHist']     # inference history of transformed state/par means
            modeHist[j,:,:] = outData['modeHist']     # inference history of transformed state/par modes
            modStates[j,:,:] = outData['modStates']   # states that have been remodeled based on the final modes of the parameters
            Nx[j,:] = outData['Nxs']                  # Number of particle sample runs
            runTime[j] = outData['runTime']           # Run Time

        except:
            print('\n\tERROR: Instability or Run Incomplete\n')
            ## Store Results ##
            for i in range(1,samps):
                muHist[j,:,i] = np.array([0,0,0, -0.1054, 2.3, 3.4, 1.9, 3.4])
                stdHist[j,:,i] = 0.01*np.ones(8)
                meanHist[j,:,i] = np.concatenate((muHist[j,:nState,i], 
                                        np.exp(muHist[j,nState:,i] + np.square(stdHist[j,nState:,i])/2)))
                modeHist[j,:,i] = np.concatenate((muHist[j,:nState,i], np.exp(muHist[j,nState:,i] 
                                        - np.square(stdHist[j,nState:,i]))))

            ### Rerun Model with Identified Parameters ###
            modStates[j,:,0] = np.concatenate((np.zeros((3,)), np.log(modeHist[j,3:,-1])))
            for i in range(1,samps):
                modStates[j,:,i] = fx(modStates[j,:,i-1], dt, exc=inpAcc[i-1]) 

        print('\tComputation Time = %d minutes and %d seconds' %(np.floor(runTime[j]), 
                                                            np.floor((runTime[j]*60) - 60*np.floor(runTime[j]))) )
        print('\tMode of Final Parameter Distributions: \n\t\txi = %.4f,\n\t\twn = %.4f,\n\t\tbeta = %.4f,\n\t\tn = %.4f,\n\t\tgamma = %.4f\n'
              %(modeHist[j,3,-1],modeHist[j,4,-1],modeHist[j,5,-1],modeHist[j,6,-1],modeHist[j,7,-1]))

saveFile = '05 - Results/outputPF_combined'
np.savez(saveFile, muHist = muHist,stdHist=stdHist, meanHist=meanHist, modeHist=modeHist, 
         modStates = modStates, Nx = Nx, runTime = runTime)

Inserting Data from Nx=10, N=50 @ Iteration 0:
	Computation Time = 282 minutes and 27 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.0501,
		wn = 2.9803,
		beta = 2.4368,
		n = 2.2610,
		gamma = 1.1628

Inserting Data from Nx=10, N=50 @ Iteration 1:
	Computation Time = 229 minutes and 30 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.0498,
		wn = 2.9908,
		beta = 2.1162,
		n = 2.1054,
		gamma = 1.1322

Inserting Data from Nx=10, N=50 @ Iteration 2:
	Computation Time = 292 minutes and 9 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.0507,
		wn = 2.9901,
		beta = 2.1195,
		n = 2.1102,
		gamma = 1.1051

Inserting Data from Nx=10, N=50 @ Iteration 3:
	Computation Time = 263 minutes and 4 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.0523,
		wn = 2.9762,
		beta = 2.5050,
		n = 2.3299,
		gamma = 1.2835

Inserting Data from Nx=10, N=50 @ Iteration 4:
	Computation Time = 223 minutes and 25 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.04

	Computation Time = 0 minutes and 0 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.8999,
		wn = 9.9732,
		beta = 29.9611,
		n = 6.6852,
		gamma = 29.9611

Inserting Data from Nx=10, N=50 @ Iteration 39:
	Computation Time = 422 minutes and 42 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.0491,
		wn = 2.9887,
		beta = 2.2409,
		n = 2.1399,
		gamma = 1.0952

Inserting Data from Nx=10, N=50 @ Iteration 40:
	Computation Time = 292 minutes and 50 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.0505,
		wn = 3.0051,
		beta = 2.0074,
		n = 2.0636,
		gamma = 1.2853

Inserting Data from Nx=10, N=50 @ Iteration 41:

	ERROR: Instability or Run Incomplete

	Computation Time = 0 minutes and 0 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.8999,
		wn = 9.9732,
		beta = 29.9611,
		n = 6.6852,
		gamma = 29.9611

Inserting Data from Nx=10, N=50 @ Iteration 42:
	Computation Time = 234 minutes and 20 seconds
	Mode of Final Parameter Distributions: 
		xi = 0.0495,